А) Какая задача решалась?

Данная статья ставила перед собой целью улучшить существующие подходы создания эмбеддингов звуковых дорожек напрямую из wave form. Результаты сравниваются на задаче верификации - о ней подробнее ниже.

Б) В чём основная идея метода и в чём её отличие от других решений? Пишите своими словами.

Основная идея метода - обработка информации о звукой дорожки непосредственно из звука - обычно используют некоторую предобработку волн для приведения ее в другой формат, будь это МЕЛ спектрограмма или что-либо еще. Эта идея не нова - уже существующие модели работающие напрямую со звуком показывают неплохой результат. Все такие модели используют первым слоем свертку с каким-то одним фиксированным набором параметров - размером ядра, шагом свертки и т.п. Такой подход не позволяет модели работать с волнами разной частоты. Авторы Yvector предлагают такое решение этой проблемы - запустить несколько сверток параллельно, каждая из которых будет обрабатывать волны своей частотности - низкой, высокой или средней.

В) Какой эксперимент ставился? Какие получились результаты и как их можно интерпретировать?

Г) Как можно использовать полученный результат? Удалось ли приблизиться к цифрам из статьи? Какие есть перспективы для развития?

Про эксперимент чуть ниже.


# Load model weights and testing dataset

In [ ]:
!git clone https://github.com/nikoryagin/YVector.git

In [2]:
%cd YVector

/content/YVector


In [ ]:
!wget https://us.openslr.org/resources/12/test-clean.tar.gz

In [ ]:
!tar -xvf test-clean.tar.gz

In [5]:
!find . -name "*.txt" -type f -delete

In [7]:
import torch
from yvector import YVectorModel
model = YVectorModel().to('cuda')
checkpoint = torch.load('/content/drive/MyDrive/vk/vk18.pth')
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

# Experiment

Чтобы повторить результаты эксперимента из статьи, я скачал тестовый датасет. В силу ограниченности ресурсов, тестовый(как и обучающий) датасет отличается от того, который использовали в статье. В чем заключается эксперимент: будем решать задачу верификации по голосу -- по двум записям с голосом, определить, принадлежит ли голос одному и тому же человеку. 
Из исходного тестового датасета будем тысячу раз генерировать по две пары звуковых дорожек - в одной паре обе дорожки принадлежат одному человеку, в другой - разным. Затем, каждая дорожка из обоих пар прогоняется через модель, таким образом, получаются эмбеддинги дорожек. В каждой паре будем считать косинус угла между эмбеддингами - мера "похожести" эмбеддингов.
На основе полученных значений косинусов и тому, принадлежит ли этот косинус паре с голосами одного человека(класс 1) или двух(класс 0), мы считаем метрики EER и minDCF - они в некотором смысле показывают, насколько в приниципе модель научилась разделять два класса. 

In [13]:
import torch.nn.functional as F
import pathlib
import os, torchaudio, random
import numpy as np

from pathlib import Path
speakers_dirs = os.listdir('LibriSpeech/test-clean')
root =  Path('LibriSpeech/test-clean')
positive_similarity = []
negative_similarity = []
model.eval()
model = model.to('cuda')
for i in range(1000):
  # Generate 2 random audiofile from (i % num_speakers)th speaker
  anc_spk_dir = root / speakers_dirs[i % len(speakers_dirs)]
  tmp_dirs = os.listdir(anc_spk_dir)
  tmp_dir = anc_spk_dir / tmp_dirs[random.randint(0, len(tmp_dirs) - 1)]
  tmp_files = os.listdir(tmp_dir)
  anchor_wave, _ = torchaudio.load(tmp_dir / tmp_files[random.randint(0, len(tmp_files) - 1)])
  pair_wave_true, _ = torchaudio.load(tmp_dir / tmp_files[random.randint(0, len(tmp_files) - 1)])

  # Generate random audiofile from other speaker
  pair_false_spk_dir = root / speakers_dirs[random.randint(0, len(speakers_dirs) - 1)]
  while pair_false_spk_dir == anc_spk_dir:
      pair_false_spk_dir = root / speakers_dirs[random.randint(0, len(speakers_dirs) - 1)]

  tmp_dirs = os.listdir(pair_false_spk_dir)
  tmp_dir = pair_false_spk_dir / tmp_dirs[random.randint(0, len(tmp_dirs) - 1)]
  tmp_files = os.listdir(tmp_dir)
  pair_wave_false, _ = torchaudio.load(tmp_dir / tmp_files[random.randint(0, len(tmp_files) - 1)])

  # Calculate embeddings for all audiofiles
  embed_anchor = model(anchor_wave.unsqueeze(0).to('cuda')).cpu().squeeze()
  embed_true = model(pair_wave_true.unsqueeze(0).to('cuda')).cpu().squeeze()
  embed_false = model(pair_wave_false.unsqueeze(0).to('cuda')).cpu().squeeze()

  # Calculate cosine similarity
  positive_similarity += [F.cosine_similarity(embed_anchor, embed_true, dim=0).detach().numpy()]
  negative_similarity += [F.cosine_similarity(embed_anchor, embed_false, dim=0).detach().numpy()]

# Utility functions

In [14]:
#from https://github.com/kaldi-asr/kaldi/blob/master/egs/sre08/v1/sid/compute_min_dcf.py
from numpy.linalg import norm
import numpy as np
from operator import itemgetter

def calculate_eer(positive_sim, negative_sim):
    target_scores = sorted(positive_sim)
    nontarget_scores = sorted(negative_sim)

    target_size = len(target_scores)
    nontarget_size = len(nontarget_scores)

    target_position = 0
    for target_position in range(target_size):
        nontarget_n = nontarget_size * target_position * 1.0 / target_size
        nontarget_position = int(nontarget_size - 1 - nontarget_n)
        if nontarget_position < 0:
            nontarget_position = 0
        if nontarget_scores[nontarget_position] < target_scores[target_position]:
            break

    threshold = target_scores[target_position]
    eer = target_position * 1.0 / target_size

    return eer, threshold

def ComputeErrorRates(scores, labels):
    # Sort the scores from smallest to largest, and also get the corresponding
    # indexes of the sorted scores.  We will treat the sorted scores as the
    # thresholds at which the the error-rates are evaluated.
    sorted_indexes, thresholds = zip(*sorted(
        [(index, threshold) for index, threshold in enumerate(scores)],
        key=itemgetter(1)))
    sorted_labels = []
    labels = [labels[i] for i in sorted_indexes]
    fnrs = []
    fprs = []

    # At the end of this loop, fnrs[i] is the number of errors made by
    # incorrectly rejecting scores less than thresholds[i]. And, fprs[i]
    # is the total number of times that we have correctly accepted scores
    # greater than thresholds[i].
    for i in range(0, len(labels)):
        if i == 0:
            fnrs.append(labels[i])
            fprs.append(1 - labels[i])
        else:
            fnrs.append(fnrs[i-1] + labels[i])
            fprs.append(fprs[i-1] + 1 - labels[i])
    fnrs_norm = sum(labels)
    fprs_norm = len(labels) - fnrs_norm

    # Now divide by the total number of false negative errors to
    # obtain the false positive rates across all thresholds
    fnrs = [x / float(fnrs_norm) for x in fnrs]

    # Divide by the total number of corret positives to get the
    # true positive rate.  Subtract these quantities from 1 to
    # get the false positive rates.
    fprs = [1 - x / float(fprs_norm) for x in fprs]
    return fnrs, fprs, thresholds

# Computes the minimum of the detection cost function.  The comments refer to
# equations in Section 3 of the NIST 2016 Speaker Recognition Evaluation Plan.
def ComputeMinDcf(fnrs, fprs, thresholds, p_target, c_miss, c_fa):
    min_c_det = float("inf")
    min_c_det_threshold = thresholds[0]
    for i in range(0, len(fnrs)):
        # See Equation (2).  it is a weighted sum of false negative
        # and false positive errors.
        c_det = c_miss * fnrs[i] * p_target + c_fa * fprs[i] * (1 - p_target)
        if c_det < min_c_det:
            min_c_det = c_det
            min_c_det_threshold = thresholds[i]
    # See Equations (3) and (4).  Now we normalize the cost.
    c_def = min(c_miss * p_target, c_fa * (1 - p_target))
    min_dcf = min_c_det / c_def
    return min_dcf, min_c_det_threshold

def calculate_minDCF(scores, labels, p_target, c_miss, c_fa):

    fnrs, fprs, thresholds = ComputeErrorRates(scores, labels)
    min_dcf, min_c_det_threshold = ComputeMinDcf(fnrs, fprs, thresholds, p_target, c_miss, c_fa)
    
    return min_dcf, min_c_det_threshold

# Results

In [15]:
total_scores = positive_similarity + negative_similarity
total_results = [1] * len(positive_similarity) + [0] * len(negative_similarity)

min_dcf2, min_c_det_threshold2 = calculate_minDCF(total_scores, total_results, 0.01, 1, 1)
min_dcf3, min_c_det_threshold3 = calculate_minDCF(total_scores, total_results, 0.001, 1, 1)

print('minDCF:0.01 {0:0.4f},{1:0.4f}'.format(min_dcf2, min_c_det_threshold2))
print('minDCF:0.001 :{0:0.4f},{1:0.4f}'.format(min_dcf3, min_c_det_threshold3))

# eer
positive_similarity = np.array(positive_similarity)
negative_similarity = np.array(negative_similarity)

eer, threshold = calculate_eer(positive_similarity, negative_similarity)
print('eer: {}%'.format(eer * 100))

minDCF:0.01 0.6050,0.6297
minDCF:0.001 :0.8250,0.8959
eer: 3.9%


**Обзор результатов** (Ответ на вопросы В и Г)

Значение основной метрики Equal Error Rate(eer) сопоставимо с результатами, опубликованными в статье(хоть я его получил и на гораздо более простом датасете).

Хочу сразу заметить несколько моментов:

1) Как уже сказано, в виду огранничености ресурсов, я обучал модель на меньшем и более простом датасете и в течение меньшего количества эпох.

2) Не все детали архитектуры были раскрыты в статье, так что в некоторых местах пришлось импровизировать(в частности, в подборе некоторых гиперпараметров)

Таким образом, я получил достойный результат, сопоставимый с SOTA моделями. Но, опять же, точно сравнить не получится, обычно на датасете, на котором я обучился и тестировался, не решают задачу верификации(этим объясняются танцы с бубнами в коде тестирования). Еще раз отмечу, что в строгих условиях ограниченных ресурсов, получился хороший результат, что говорит о фундаментальной идейной "правильности" архитектуры YVector.

Полученные результаты можно использовать во всех областях обработки и синтеза звука - значения метрик, полученные для задачи верификации позволяют сделать предположение, что эмбеддинги YVector несут в себе больше информации. В качестве перспектив развития можно предложить исследования в области архитектуры - улучшится ли результат если добавить больше параллельных сверток? Возможны ли другие способы обработки кроме сверток? Self-attention?